In [1]:
#| default_exp reading
%load_ext autoreload
%autoreload 2

import sys
from fastcore.xtras import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Leitura
> Este módulo concentra funções para a leitura dos diversos arquivos que compõe a base de dados

* `STEL` - Serviços Privados de Telecomunicações
* `RADCOM` - Serviço de Radiodifusão Comunitária
* `MOSAICO` - Demais serviços de Radiodifusão, e.g. TV, RTV, FM, AM, etc.
* `AERONAUTICA` - Consolidação de diversas bases de dados públicas da aeronáutica 

In [1]:
#| export
from typing import Union, Tuple
from pathlib import Path

import pandas as pd
from pyarrow import ArrowInvalid
import pyodbc
from pymongo import MongoClient

from anateldb.updates import (
update_mosaico, update_stel, update_radcom, update_base
)

In [3]:
#| export
def _read_df(folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Lê o dataframe formado por folder / stem.[parquet.gzip | fth | xslx]"""
    file = Path(f"{folder}/{stem}.parquet.gzip")
    try:
        df = pd.read_parquet(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{folder}/{stem}.fth")
        try:
            df = pd.read_feather(file)
        except (ArrowInvalid, FileNotFoundError):
            file = Path(f"{folder}/{stem}.xlsx")
            try:
                df = pd.read_excel(file, engine="openpyxl", sheet_name="DataBase")
            except Exception as e:
                raise ValueError(f"Error when reading {file}") from e
    return df

## MOSAICO
> O mosaico atualmente é composto por 2 bases complementares originárias de um banco `MongoDB`: 

-  `Estações`
- `Plano Básico`

In [4]:
#| export
def read_mosaico(folder: Union[str, Path], # Pasta onde ler/salvar os dados
                 update: bool = False, # Atualiza os dados caso `True`
) -> pd.DataFrame: # Dataframe com os dados do mosaico
    """Lê o banco de dados salvo localmente do MOSAICO e opcionalmente o atualiza."""
    return update_mosaico(folder) if update else _read_df(folder, "mosaico")

In [5]:
#| hide
folder = Path.cwd().parent / 'dados'

In [6]:
#|echo: false
# from pandas_profiling import ProfileReport
# mosaico = read_mosaico(folder)
# mosaico['Frequência'] = mosaico['Frequência'].astype('category')
# mosaico_profile = ProfileReport(mosaico, config_file='report_config.yaml', title='MOSAICO')
# mosaico_profile.to_notebook_iframe()

##  RADCOM

In [7]:
#| export
def read_radcom(
    conn: pyodbc.Connection, # Objeto de conexão de banco
    folder: Union[str, Path], # Pasta onde ler/salvar os dados
    update: bool = False # Atualiza os dados caso `True`
                
) -> pd.DataFrame: # Dataframe com os dados de RADCOM
    """Lê o banco de dados salvo localmente de RADCOM. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO05 caso `update = True` ou não exista o arquivo local"""
    return update_radcom(folder) if update else _read_df(folder, "radcom")

In [8]:
#|echo: false
# radcom = read_radcom(folder)
# radcom['Frequência'] = radcom['Frequência'].astype('category')
# radcom_profile = ProfileReport(radcom, config_file='report_config.yaml', title='RADCOM')
# radcom_profile.to_notebook_iframe()

## STEL

In [9]:
#| export
def read_stel(
  conn: pyodbc.Connection, # Objeto de conexão de banco
  folder: Union[str, Path], # Pasta onde ler/salvar os dados
  update: bool = False # Atualiza os dados caso `True`
              
) -> pd.DataFrame: # Dataframe com os dados do STEL
    """Lê o banco de dados salvo localmente do STEL. 
       Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO05
      caso `update = True` ou não exista o arquivo local"""
    return update_stel(folder) if update else _read_df(folder, "stel")

In [10]:
#|echo: false
# stel = read_stel(folder)
# stel['Frequência'] = stel['Frequência'].astype('category')
# stel_profile = ProfileReport(stel, config_file='report_config.yaml', title='STEL')
# stel_profile.to_notebook_iframe()

## Bases Externas da Aeronáutica

In [11]:
#| export
def read_icao(folder: Union[str, Path], # Pasta onde ler/salvar os dados
              update: bool = False, # Atualiza os dados caso `True`
) -> pd.DataFrame: # Dataframe com os dados do ICAO
    """Lê a base de dados do Frequency Finder e Canalização VOR/ILS/DME"""
    if update:
        # TODO: atualizar a base de dados do Frequency Finder e Canalização VOR/ILS/DME
        # update_icao(folder)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    return _read_df(folder, "icao")


In [12]:
#| echo: false
# icao = read_icao(folder)
# icao['Frequency'] = icao['Frequency'].astype('category')
# icao_profile = ProfileReport(icao, config_file='report_config.yaml', title='ICAO')
# icao_profile.to_notebook_iframe()

In [13]:
#| export
def read_aisw(folder: Union[str, Path], # Pasta onde ler/salvar os dados
              update: bool = False, # Atualiza os dados caso `True`
) -> pd.DataFrame: # Dataframe com os dados do AISWEB
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    if update:
        # TODO: Atualizar a base de dados do AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME
        # update_pmec(folder)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    return _read_df(folder, "aisw")



In [14]:
#| echo: false
# aisw = read_aisw(folder)
# aisw['Frequency'] = aisw['Frequency'].astype('category')
# aisw_profile = ProfileReport(aisw, config_file='report_config.yaml', title='AISWEB')
# aisw_profile.to_notebook_iframe()

In [15]:
#| export
def read_aisg(folder: Union[str, Path], # Pasta onde ler/salvar os dados
update: bool = False, # Atualiza os dados caso `True`
) -> pd.DataFrame: # Dataframe com os dados do GEOAISWEB
    """Fontes da informação: GEOAISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    if update:
        # TODO: Atualizar a base de dados do GEOAISWEB
        # update_geo(folder)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    return _read_df(folder, "aisg")

In [16]:
#| echo: false
# aisg = read_aisg(folder)
# aisg['Frequency'] = aisg['Frequency'].astype('category')
# aisg_profile = ProfileReport(aisg, config_file='report_config.yaml', title='GEOAISWEB')
# aisg_profile.to_notebook_iframe()

In [17]:
#| export
def read_aero(
    folder: Union[str, Path], # Pasta onde ler/salvar os dados
update: bool = False, # Atualiza os dados caso `True`
) -> pd.DataFrame: # Dataframe com os dados mesclados das 3 bases da Aeronáutica anteriores
    """Lê os arquivos de dados da aeronáutica e retorna os registros comuns e únicos"""
    if update:
        # TODO: Atualizar a base de dados do GEOAISWEB
        # update_geo(folder)
        raise NotImplementedError(
            "Atualização programática das bases de Dados da Aeronáutica não implementada"
        )
    
    return _read_df(folder, "aero")

In [18]:
#| echo: false
# from pandas_profiling import ProfileReport
# aero= read_aero(folder)
# aero['Frequency'] = aero['Frequency'].astype('category')
# aero_profile = ProfileReport(aero, config_file='report_config.yaml', title='AERONAUTICA')
# aero_profile.to_notebook_iframe()

## Base Consolidada

In [2]:
#| export
def read_base(
    conn: pyodbc.Connection, # Objeto de conexão de banco
    clientMongoDB: MongoClient, # Ojeto de conexão com o MongoDB
    folder: Union[str, Path], 
    update: bool = False
) -> pd.DataFrame:
    """Lê a base de dados e opcionalmente a atualiza antes da leitura"""
    return update_base(conn, clientMongoDB, folder) if update else _read_df(folder, "base")

In [20]:
#| echo: false
# from pandas_profiling import ProfileReport
# base = read_base(folder)
# base['Frequência'] = base['Frequência'].astype('category')
# base_profile = ProfileReport(base, config_file='report_config.yaml', title='Base Consolidada')
# base_profile.to_notebook_iframe()